In [ ]:
from pygraphblas import *
import pygraphblas.descriptor
import csv
import sys
import logging
import glob

from data_loader import DataLoader


In [ ]:
# Setup logger
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-5s %(message)s'))
log = logging.getLogger(__name__)
log.propagate = False
log.addHandler(handler)
log.setLevel(logging.INFO)

### Load data

In [ ]:
data_dir = 'sf1k-converted/'
loader = DataLoader(data_dir)

vertices, mappings, matrices = loader.load_all_csvs()

### Queries

In [ ]:
# Query 1
def shortest_distance_over_frequent_communication_paths(matrices, threshhold):
    hasCreatorTransposed = matrices['hasCreator'].transpose()
    
    
    personA_to_comment2 = hasCreatorTransposed @ matrices['replyOf']
    person_to_person = personA_to_comment2.mxm(matrices['hasCreator'], mask=matrices['knows'])
    person_to_person_mutual = person_to_person + person_to_person.transpose()
    person_to_person_mutual_filtered = person_to_person_mutual.select('<', threshhold)    
    person_to_person_mutual_filtered = person_to_person_mutual_filtered.pattern() # every element will be boolean
    # TODO: BFS
    
    return person_to_person_mutual_filtered
    
    
    
    
    
    

In [ ]:
x = shortest_distance_over_frequent_communication_paths(matrices, 10)

In [ ]:
x.to_string()